### HISC-Lasso: Calcul du score RED pour $m=50$ et MCAcc

In [58]:
# install pyhsic lasso
!pip install pyHSICLasso

In [59]:
import numpy as np
import pandas as pd
from pyHSICLasso import HSICLasso
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score

In [61]:
# data process
X = pd.read_csv("ar10p.csv")
n, p = X.shape
p -= 1
Y = X.iloc[:, p].values
X = X.iloc[:, :p].values

#standardisation des données
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
rep = 50
features_index = np.arange(10, 51, 10)
top_m = features_index[-1]
MCAcc = np.zeros((len(features_index), rep))

In [ ]:
for i in range(rep):
    print(i)
    x_index = np.random.choice(n, size=int(n * 0.8), replace=False)
    X_train, Y_train = X[x_index], Y[x_index]
    X_test, Y_test = np.delete(X, x_index, axis=0), np.delete(Y, x_index)

    # Sélection de variables avec HSIC Lasso
    hsic_lasso = HSICLasso()
    hsic_lasso.input(X_train, Y_train,kernelX="Gaussian",sigmaX=0.5,kernelY="Delta")

    hsic_lasso.classification(num_feat=top_m)
    ModelFeatureIndex = hsic_lasso.get_index()

    for m_index, nb_features in enumerate(features_index):
        x_features = ModelFeatureIndex[:nb_features]

        # Modèle de classification
        kernel = 1.0 * RBF(0.1)
        model = GaussianProcessClassifier(kernel=kernel,multi_class="one_vs_one")
        model.fit(X_train[:, x_features], Y_train)

        # Prédiction et évaluation
        y_pred = model.predict(X_test[:, x_features])
        MCAcc[m_index, i] = accuracy_score(Y_test, y_pred)

0
Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.


/usr/local/lib/python3.11/dist-packages/pyHSICLasso/api.py:107: RuntimeWarning: B 20 must be an exact divisor of the number of samples 104. Number of blocks 5.2 will be approximated to 5.
  warnings.warn(msg, RuntimeWarning)


In [ ]:
# Résultats statistiques
rmean = MCAcc.mean(axis=1)

# Visualisation
import matplotlib.pyplot as plt
plt.plot(features_index, rmean, color='cyan', linestyle='-', marker='o')
plt.ylim(0, 1)
plt.show()

In [ ]:
# Sauvegarde des résultats
np.savetxt("hsic_sigma_value.csv", MCAcc, delimiter=",")